## Import necessary libraries

In [1]:
import re
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV


## Load training and test data

In [2]:

df = pd.read_csv('/kaggle/input/playground-series-s4e9/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s4e9/test.csv')


In [3]:
df.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [4]:
df_test.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,White,Beige,None reported,Yes
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,395.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,8-Speed A/T,Silver,Black,None reported,Yes
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,3.5L V6 24V PDI DOHC Twin Turbo,10-Speed Automatic,White,Ebony,None reported,NaN
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,2.0 Liter TFSI,Automatic,Silician Yellow,Black,None reported,NaN
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,252.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            125690 non-null  int64 
 1   brand         125690 non-null  object
 2   model         125690 non-null  object
 3   model_year    125690 non-null  int64 
 4   milage        125690 non-null  int64 
 5   fuel_type     122307 non-null  object
 6   engine        125690 non-null  object
 7   transmission  125690 non-null  object
 8   ext_col       125690 non-null  object
 9   int_col       125690 non-null  object
 10  accident      124058 non-null  object
 11  clean_title   111451 non-null  object
dtypes: int64(3), object(9)
memory usage: 11.5+ MB


## Data Preprocessing

In [6]:
df['fuel_type'].fillna('Electricity', inplace=True)
df['clean_title'].fillna('No', inplace=True)
df['accident'].fillna('None reported', inplace=True)



In [7]:
df_test['fuel_type'].fillna('Electricity', inplace=True)
df_test['clean_title'].fillna('No', inplace=True)
df_test['accident'].fillna('None reported', inplace=True)



## Feature extraction from engine

In [8]:

def extract_hp(engine):
    match = re.search(r'(\d+(\.\d+)?)HP', engine)
    return float(match.group(1)) if match else None

def extract_displacement(engine):
    match = re.search(r'(\d+\.\d+)L|(\d+\.\d+) Liter', engine)
    if match:
        # Check if the first group matched, otherwise use the second group
        return float(match.group(1) or match.group(2))

def extract_engine_type(engine):
    match = re.search(r'(V\d+|I\d+|Flat \d+|Straight \d+)', engine)
    return match.group(1) if match else None

def extract_cylinder_count(engine):
    match = re.search(r'(\d+) Cylinder', engine)
    return int(match.group(1)) if match else None

def extract_fuel_type(engine):
    fuel_types = ['Gasoline', 'Diesel', 'Electric', 'Hybrid', 'Flex Fuel']
    for fuel in fuel_types:
        if fuel in engine:
            return fuel
    return None


In [9]:

df['Horsepower'] = df['engine'].apply(extract_hp)
df['Displacement'] = df['engine'].apply(extract_displacement)
df['Engine Type'] = df['engine'].apply(extract_engine_type)
df['Cylinder Count'] = df['engine'].apply(extract_cylinder_count)
df['Fuel Type'] = df['engine'].apply(extract_fuel_type)

In [10]:
df_test['Horsepower'] = df_test['engine'].apply(extract_hp)
df_test['Displacement'] = df_test['engine'].apply(extract_displacement)
df_test['Engine Type'] = df_test['engine'].apply(extract_engine_type)
df_test['Cylinder Count'] = df_test['engine'].apply(extract_cylinder_count)
df_test['Fuel Type'] = df_test['engine'].apply(extract_fuel_type)

## Handle missing values in test data similarly

In [11]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
df[['Horsepower', 'Cylinder Count']] = imputer.fit_transform(df[['Horsepower', 'Cylinder Count']])
df_test[['Horsepower', 'Cylinder Count']] = imputer.transform(df_test[['Horsepower', 'Cylinder Count']])



In [12]:
columns_to_drop = ['engine', 'model']
df.drop(columns=[col for col in columns_to_drop if col in df.columns], inplace=True)
df_test.drop(columns=[col for col in columns_to_drop if col in df_test.columns], inplace=True)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              188533 non-null  int64  
 1   brand           188533 non-null  object 
 2   model_year      188533 non-null  int64  
 3   milage          188533 non-null  int64  
 4   fuel_type       188533 non-null  object 
 5   transmission    188533 non-null  object 
 6   ext_col         188533 non-null  object 
 7   int_col         188533 non-null  object 
 8   accident        188533 non-null  object 
 9   clean_title     188533 non-null  object 
 10  price           188533 non-null  int64  
 11  Horsepower      188533 non-null  float64
 12  Displacement    181763 non-null  float64
 13  Engine Type     88970 non-null   object 
 14  Cylinder Count  188533 non-null  float64
 15  Fuel Type       156987 non-null  object 
dtypes: float64(3), int64(4), object(9)
memory usage: 23.0+ M

In [14]:
df["accident"] = df["accident"].replace({'At least 1 accident or damage reported':1,"None reported":0}) 
df["accident"].value_counts()

accident
0    146966
1     41567
Name: count, dtype: int64

In [15]:
df_test["accident"] = df_test["accident"].replace({'At least 1 accident or damage reported':1,"None reported":0}) 
df_test["accident"].value_counts()

accident
0    97895
1    27795
Name: count, dtype: int64

In [16]:
df["clean_title"] = df["clean_title"].replace({"Yes":1,"No":0})
df["clean_title"].value_counts()

clean_title
1    167114
0     21419
Name: count, dtype: int64

In [17]:
df_test["clean_title"] = df["clean_title"].replace({"Yes":1,"No":0})
df_test["clean_title"].value_counts()

clean_title
1    111435
0     14255
Name: count, dtype: int64

In [18]:
df.drop(['Engine Type'], axis=1, inplace=True)
df_test.drop(['Engine Type'], axis=1, inplace=True)

In [19]:
df.head()

,id,brand,model_year,milage,fuel_type,transmission,ext_col,int_col,accident,clean_title,price,Horsepower,Displacement,Cylinder Count,Fuel Type
0,0,MINI,2007,213000,Gasoline,A/T,Yellow,Gray,0,1,4200,172.0,1.6,4.0,Gasoline
1,1,Lincoln,2002,143250,Gasoline,A/T,Silver,Beige,1,1,4999,252.0,3.9,8.0,Gasoline
2,2,Chevrolet,2002,136731,E85 Flex Fuel,A/T,Blue,Gray,0,1,13900,320.0,5.3,8.0,Flex Fuel
3,3,Genesis,2017,19500,Gasoline,Transmission w/Dual Shift Mode,Black,Black,0,1,45000,420.0,5.0,8.0,Gasoline
4,4,Mercedes-Benz,2021,7388,Gasoline,7-Speed A/T,Black,Beige,0,1,97500,208.0,2.0,4.0,Gasoline


In [20]:
df['transmission'] = df['transmission'].str.strip().str.lower()

transmission_mapping = {

    '6-speed a/t': 'Automatic',
    '8-speed automatic': 'Automatic',
    'automatic': 'Automatic',
    '7-speed a/t': 'Automatic',
    'a/t': 'Automatic',
    '8-speed a/t': 'Automatic',
    'transmission w/dual shift mode': 'Automatic',
    '9-speed automatic': 'Automatic',
    '10-speed automatic': 'Automatic',
    '1-speed a/t': 'Automatic',
    '2-speed a/t': 'Automatic',
    '2-speed automatic': 'Automatic',
    '4-speed a/t': 'Automatic',
    '5-speed automatic': 'Automatic',
    '4-speed automatic': 'Automatic',
    '6-speed automatic': 'Automatic', 
    '9-speed a/t': 'Automatic',        
    '10-speed a/t': 'Automatic',      
    '7-speed automatic': 'Automatic',  
    '6-speed electronically controlled automatic with o': 'Automatic',
    'single-speed fixed gear': 'Automatic',
    '7-speed dct automatic': 'Automatic',
    '10-speed automatic with overdrive': 'Automatic',
    'automatic, 9-spd 9g-tronic': 'Automatic',
    'automatic, 8-spd': 'Automatic',
    'automatic, 8-spd sport w/sport & manual modes': 'Automatic',
    'automatic, 8-spd pdk dual-clutch': 'Automatic',
    'automatic, 8-spd m steptronic w/drivelogic, sport & manual modes': 'Automatic',
    'automatic, 8-spd dual-clutch': 'Automatic',
    'transmission overdrive switch': 'Automatic',  
    
 
    '7-speed automatic with auto-shift': 'Tiptronic',
    '5-speed a/t': 'Tiptronic',
    '7-speed a/t tiptronic': 'Tiptronic',  
    '8-speed at': 'Tiptronic',
    '8-speed a/t': 'Tiptronic',
    
 
    '6-speed m/t': 'Manual',
    '7-speed m/t': 'Manual',
    '6-speed manual': 'Manual',
    '5-speed m/t': 'Manual',
    'manual': 'Manual',
    '7-speed manual': 'Manual',
    '8-speed manual': 'Manual',
    'm/t': 'Manual',
    '6 speed at/mt': 'Manual',
    '6 speed mt': 'Manual',

 
    'automatic cvt': 'Variator',
    'cvt transmission': 'Variator',
    'cvt-f': 'Variator',
    
  
    'variable': 'Variator',
    'f': 'Other',                        
    '7-speed': 'Other',                 
    '6-speed': 'Other',                  
    '2': 'Other',                       
    '–': 'Other',
    'scheduled for or in production': 'Other'
}


df['transmission'] = df['transmission'].replace(transmission_mapping)


print(df['transmission'].value_counts())

transmission
Automatic                            142801
Tiptronic                             25054
Manual                                16415
Variator                               2900
8-speed automatic with auto-shift       549
1-speed automatic                       365
Other                                   305
6-speed automatic with auto-shift       111
9-speed automatic with auto-shift        33
Name: count, dtype: int64


In [21]:
def simplify_transmission(transmission):
    if 'Automatic' in transmission:
        return 'Automatic'
    elif 'Manual' in transmission:
        return 'Manual'
    elif 'Tiptronic' in transmission:
        return 'Tiptronic'
    elif 'Variator' in transmission:
        return 'Variator'
    else:
        return 'Other'


df['transmission'] = df['transmission'].apply(simplify_transmission)


df['transmission'].value_counts()

transmission
Automatic    142801
Tiptronic     25054
Manual        16415
Variator       2900
Other          1363
Name: count, dtype: int64

In [22]:
df_test['transmission'] = df_test['transmission'].replace(transmission_mapping)


print(df_test['transmission'].value_counts())

transmission
A/T                                                   33132
8-Speed A/T                                           13941
Transmission w/Dual Shift Mode                        12824
6-Speed A/T                                           12061
6-Speed M/T                                            7865
7-Speed A/T                                            7374
Automatic                                              7175
8-Speed Automatic                                      5546
10-Speed A/T                                           5303
9-Speed A/T                                            2584
5-Speed A/T                                            2241
10-Speed Automatic                                     2104
6-Speed Automatic                                      1891
4-Speed A/T                                            1666
5-Speed M/T                                            1639
9-Speed Automatic                                      1553
CVT Transmission           

In [23]:
def simplify_transmission(transmission):
    if 'Automatic' in transmission:
        return 'Automatic'
    elif 'Manual' in transmission:
        return 'Manual'
    elif 'Tiptronic' in transmission:
        return 'Tiptronic'
    elif 'Variator' in transmission:
        return 'Variator'
    else:
        return 'Other'


df_test['transmission'] = df_test['transmission'].apply(simplify_transmission)


df_test['transmission'].value_counts()

transmission
Other        104780
Automatic     20650
Manual          260
Name: count, dtype: int64

In [24]:
df["transmission"] = df["transmission"].replace({'Automatic':1,
                                                'Tiptronic':2,
                                                'Manual':3,
                                                'Variator':4,
                                                'Other':5})
df["transmission"].value_counts()

transmission
1    142801
2     25054
3     16415
4      2900
5      1363
Name: count, dtype: int64

In [25]:
df_test["transmission"] = df["transmission"].replace({'Automatic':1,
                                                'Tiptronic':2,
                                                'Manual':3,
                                                'Variator':4,
                                                'Other':5})
df_test["transmission"].value_counts()

transmission
1    95171
2    16706
3    10983
4     1928
5      902
Name: count, dtype: int64

In [26]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              125690 non-null  int64  
 1   brand           125690 non-null  object 
 2   model_year      125690 non-null  int64  
 3   milage          125690 non-null  int64  
 4   fuel_type       125690 non-null  object 
 5   transmission    125690 non-null  int64  
 6   ext_col         125690 non-null  object 
 7   int_col         125690 non-null  object 
 8   accident        125690 non-null  int64  
 9   clean_title     125690 non-null  int64  
 10  Horsepower      125690 non-null  float64
 11  Displacement    121252 non-null  float64
 12  Cylinder Count  125690 non-null  float64
 13  Fuel Type       104675 non-null  object 
dtypes: float64(3), int64(6), object(5)
memory usage: 13.4+ MB


In [27]:
df["Fuel Type"].value_counts()

Fuel Type
Gasoline     139734
Electric       7566
Flex Fuel      5168
Diesel         4119
Hybrid          400
Name: count, dtype: int64

In [28]:

df.drop('Fuel Type',axis=1,inplace=True)
df_test.drop('Fuel Type',axis=1,inplace=True)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              188533 non-null  int64  
 1   brand           188533 non-null  object 
 2   model_year      188533 non-null  int64  
 3   milage          188533 non-null  int64  
 4   fuel_type       188533 non-null  object 
 5   transmission    188533 non-null  int64  
 6   ext_col         188533 non-null  object 
 7   int_col         188533 non-null  object 
 8   accident        188533 non-null  int64  
 9   clean_title     188533 non-null  int64  
 10  price           188533 non-null  int64  
 11  Horsepower      188533 non-null  float64
 12  Displacement    181763 non-null  float64
 13  Cylinder Count  188533 non-null  float64
dtypes: float64(3), int64(7), object(4)
memory usage: 20.1+ MB


In [30]:
df['Displacement'].unique()

array([1.6 , 3.9 , 5.3 , 5.  , 2.  , 3.  , 2.7 , 3.6 ,  nan, 6.2 , 4.4 ,
       4.  , 4.6 , 5.2 , 3.7 , 5.7 , 6.7 , 3.5 , 2.8 , 5.5 , 5.4 , 6.8 ,
       3.8 , 2.5 , 3.4 , 2.4 , 3.3 , 2.3 , 3.2 , 4.7 , 1.8 , 6.  , 4.8 ,
       5.6 , 4.3 , 4.2 , 1.5 , 6.4 , 8.3 , 6.3 , 2.1 , 1.4 , 4.5 , 1.3 ,
       2.9 , 1.7 , 6.6 , 2.2 , 6.1 , 7.3 , 6.5 , 0.65, 6.75, 7.  , 1.  ,
       5.9 , 8.1 , 5.8 , 1.2 , 8.4 , 8.  , 7.4 ])

In [31]:
df['Displacement'].fillna(value=df['Displacement'].mean(),inplace=True)

In [32]:
df.isna().sum()

id                0
brand             0
model_year        0
milage            0
fuel_type         0
transmission      0
ext_col           0
int_col           0
accident          0
clean_title       0
price             0
Horsepower        0
Displacement      0
Cylinder Count    0
dtype: int64

In [33]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125690 entries, 0 to 125689
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   id              125690 non-null  int64  
 1   brand           125690 non-null  object 
 2   model_year      125690 non-null  int64  
 3   milage          125690 non-null  int64  
 4   fuel_type       125690 non-null  object 
 5   transmission    125690 non-null  int64  
 6   ext_col         125690 non-null  object 
 7   int_col         125690 non-null  object 
 8   accident        125690 non-null  int64  
 9   clean_title     125690 non-null  int64  
 10  Horsepower      125690 non-null  float64
 11  Displacement    121252 non-null  float64
 12  Cylinder Count  125690 non-null  float64
dtypes: float64(3), int64(6), object(4)
memory usage: 12.5+ MB


In [34]:
df_test['Displacement'].fillna(value=df_test['Displacement'].mean(),inplace=True)

In [35]:
df_test['fuel_type'].value_counts()

fuel_type
Gasoline          110533
Hybrid              4676
E85 Flex Fuel       3523
Electricity         3383
Diesel              2686
–                    538
Plug-In Hybrid       337
not supported         14
Name: count, dtype: int64

In [36]:
# Encoding categorical variables
categorical_columns = ['brand', 'fuel_type', 'ext_col', 'int_col']
le = LabelEncoder()

for col in categorical_columns:
    if col in df.columns:
        df[col] = le.fit_transform(df[col])
        df_test[col] = le.transform(df_test[col])


## Prepare training and testing datasets

In [37]:
scaler = StandardScaler()
df[['Horsepower', 'Displacement', 'Cylinder Count']] = scaler.fit_transform(df[['Horsepower', 'Displacement', 'Cylinder Count']])
df_test[['Horsepower', 'Displacement', 'Cylinder Count']] = scaler.transform(df_test[['Horsepower', 'Displacement', 'Cylinder Count']])

In [38]:
X = df.drop(columns=['price']) 
y = df['price']  

In [39]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)


## Build Model with LGBM

In [40]:
lgbm_params = {
    'subsample': 0.95, 
    'reg_lambda': 0.005623413251903491, 
    'reg_alpha': 1.0, 
    'num_leaves': 570, 
    'n_estimators': 550, 
    'min_data_in_leaf': 135, 
    'min_child_weight': 0.02, 
    'max_depth': 13, 
    'learning_rate': 0.015, 
    'feature_fraction': 0.85, 
    'colsample_bytree': 0.9, 
    'cat_smooth': 50, 
    'bagging_freq': 9, 
    'bagging_fraction': 0.85
}
lgbm_model = lgb.LGBMRegressor(**lgbm_params, random_state=42, verbose=-1)
lgbm_model.fit(X_train, y_train)

LGBMRegressor(bagging_fraction=0.85, bagging_freq=9, cat_smooth=50,
              colsample_bytree=0.9, feature_fraction=0.85, learning_rate=0.015,
              max_depth=13, min_child_weight=0.02, min_data_in_leaf=135,
              n_estimators=550, num_leaves=570, random_state=42, reg_alpha=1.0,
              reg_lambda=0.005623413251903491, subsample=0.95, verbose=-1)

In [41]:
from catboost import CatBoostRegressor

In [42]:
catb_params = {
    'iterations': 1500, 
    'learning_rate': 0.010011982648647914, 
    'depth': 10, 
    'l2_leaf_reg': 9.557443197213443, 
    'bootstrap_type': 'MVS'
}
cat_features = [X_train.columns.get_loc(col) for col in categorical_columns]
catb_model = CatBoostRegressor(**catb_params, cat_features=cat_features, random_state=42, verbose=500)
catb_model.fit(X_train, y_train)

0:	learn: 79748.3584224	total: 262ms	remaining: 6m 32s
500:	learn: 72678.6728266	total: 1m 31s	remaining: 3m 2s
1000:	learn: 72065.2319785	total: 2m 39s	remaining: 1m 19s
1499:	learn: 71820.5343069	total: 3m 25s	remaining: 0us


In [43]:
lgbm_preds = lgbm_model.predict(X_valid)
catb_preds = catb_model.predict(X_valid)

In [44]:
ensemble_preds = (lgbm_preds + catb_preds) / 2
rmse_ensemble = mean_squared_error(y_valid, ensemble_preds, squared=False)
print(f'Ensemble RMSE: {rmse_ensemble}')

Ensemble RMSE: 67914.81224312125


In [45]:
X_test = df_test  
lgbm_test_preds = lgbm_model.predict(X_test)
catb_test_preds = catb_model.predict(X_test)

In [46]:
ensemble_test_preds = (lgbm_test_preds + catb_test_preds) / 2

In [47]:
submission = pd.DataFrame({
    'id': df_test['id'],  
    'price': ensemble_test_preds
})

In [48]:
submission.to_csv('submission.csv', index=False)
print("Submission file created!")

Submission file created!
